# Example usages of **chat** and **service** interfaces
This notebook shows simple usages of MemoryScope's **chat** and **service** interfaces, along with its main features.

To run this notebook, follow the **Installation** guidelines in Readme, and start the Docker image.


## Initiate a MemoryScope instance
First, we need to specify a configuration and initiate a MemoryScope instance.

In [35]:
import sys
sys.path.append(".")
from memoryscope import MemoryScope, Arguments
arguments = Arguments(
    language="cn",
    human_name="用户",
    assistant_name="AI",
    logger_to_screen=False,
    memory_chat_class="api_memory_chat",
    generation_backend="dashscope_generation",
    generation_model="qwen2-72b-instruct",
    embedding_backend="dashscope_embedding",
    embedding_model="text-embedding-v2",
    rank_backend="dashscope_rank",
    rank_model="gte-rerank",
    enable_ranker=True,
    worker_params={"get_reflection_subject": {"reflect_num_questions": 3}}
)

ms = MemoryScope(arguments=arguments)


## Chat without memory
ms comes with a default **chat** interface, so it's very easy to start chatting, just as what you'll do with any LLM chatbot.

In [36]:
memory_chat = ms.default_memory_chat
memory_chat.run_service_operation("delete_all")
response = memory_chat.chat_with_memory(query="我的爱好是弹琴。")
print("回答1：\n" + response.message.content)

回答1：
那真是一个优雅的爱好！弹琴不仅能够陶冶情操，还能提升音乐素养和手指灵活性。你最喜欢弹奏哪种类型的曲子呢？


You can choose to chat with or without multi-round conversation contexts. However, since *Memory Consolidation* has not been called, there's no memory pieces in the system yet.

In [37]:
response = memory_chat.chat_with_memory(query="你知道我的乐器爱好是什么吗？")
print("回答2：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="你知道我的乐器爱好是什么吗？",
                                        history_message_strategy=None)
print("回答3：\n" + response.message.content)

回答2：
根据您之前的回答，您的乐器爱好是弹琴。
回答3：
对不起，我无法记住或了解特定用户的具体个人信息，如乐器爱好，除非您之前已通过相应渠道告诉我。如果您愿意分享，我很乐意了解您的乐器爱好。


## Memory Consolidation
Now, we do a bit more chatting and try out **Memory Consolidation**.

In [38]:
response = memory_chat.chat_with_memory(query="我在阿里巴巴干活")
print("回答4：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="今天下午吃什么水果好？")
print("回答5：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="我喜欢吃西瓜。")
print("回答6：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="帮我写一句给朋友的生日祝福语，简短一点。")
print("回答7：\n" + response.message.content)

回答4：
阿里巴巴是一家知名的中国跨国科技公司，主要从事电子商务、零售、互联网和技术领域。在阿里巴巴工作，您可能涉及的范围很广，包括电商运营、云计算、金融科技、人工智能等多个方面。这是一家非常注重创新和科技驱动的企业，希望您在那儿的工作经历丰富且充满挑战！
回答5：
今天吃水果是个不错的选择！考虑到个人口味和营养均衡，您可以选择当季新鲜的水果。比如，如果是夏季，西瓜、蜜瓜、桃子和李子都是既清爽又多汁的好选择，它们能帮助消暑解渴，补充水分和维生素。如果偏好柑橘类，橙子和葡萄柚也是很好的选择，富含维生素C，有助于提升免疫力。最终选择哪种，还请根据自己的健康状况和喜好来决定。
回答6：
西瓜是夏季里非常受欢迎的水果，它含水量高，能够很好地帮助身体补水，同时含有丰富的维生素A、C和一些矿物质，对于解暑降温非常有帮助。它的甜味来自天然果糖，适合大多数人群食用。享受您的西瓜时光，既美味又健康！
回答7：
"生日快乐，愿你的每一天都如蛋糕般甜蜜，笑容比烛光更灿烂！"


In [39]:
memory_service = ms.default_memory_service
memory_service.init_service()
result = memory_service.consolidate_memory()
print(f"consolidate_memory result={result}")

/Users/tiancheng/anaconda3/envs/memory/lib/python3.12/site-packages/elasticsearch/helpers/vectorstore/_sync/vectorstore.py:315: RuntimeWarning: coroutine 'AsyncDenseVectorStrategy.before_index_creation' was never awaited
  self.retrieval_strategy.before_index_creation(


consolidate_memory result=[MEMORY ACTIONS]:
new 用户有乐器爱好。
new 用户在阿里巴巴工作。
new 用户喜欢吃西瓜。
modified 用户爱好弹琴。


**Memory Consolidation** extracted 3 *observations* out of the 7 chat messages from the user, with the uninformative ones being filtered out.

We try more cases to test its time awareness and the ability to filter out fictitious contents from the user.

In [40]:
response = memory_chat.chat_with_memory(query="假如我去京东工作，前景怎么样？")
print("回答8：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="记一下，下周我准备去北京出差")
print("回答9：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="我同学李亚平现在在亚马逊工作，他下个月回上海，我要和他吃个饭")
print("回答10：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="小亮是我最好的朋友，他决定去山西上大学。以这个为开头写一个80字的微剧本。")
print("回答11：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="SMCI是什么公司，做什么的？")
print("回答12：\n" + response.message.content)

回答8：
京东作为中国领先的电商平台之一，拥有广泛的业务范围和良好的行业声誉。加入京东工作，您可能会享受到以下几方面的前景优势：

1. **职业发展**：京东提供多元化的职业路径和晋升机会，您可以在电商、物流、科技、金融等多个领域找到适合自己的位置和发展空间。

2. **技术与创新**：京东持续投入于技术创新，尤其是在人工智能、大数据、云计算等方面，为员工提供了接触和学习前沿技术的机会。

3. **企业文化和福利**：京东注重员工关怀，提供有竞争力的薪酬福利体系和良好的工作环境，有利于提升员工的工作满意度和生活品质。

4. **行业影响力**：作为行业巨头，京东的从业经历对您的职业生涯将是一大亮点，有助于增强个人品牌和未来的职业选择。

当然，具体前景还取决于您的职位、能力、以及行业动态等因素。建议您深入了解目标岗位的具体要求和发展路径，同时关注行业趋势，为自己的职业规划做好准备。
回答9：
好的，已记录您计划下周去北京出差。请随时告诉我如果您需要关于北京的出行建议或信息帮助。
回答10：
已记录您下个月将与在亚马逊工作的同学李亚平在上海见面吃饭的安排。希望你们有个愉快的聚餐！如果有需要推荐餐厅或安排建议，请随时告诉我。
回答11：
【场景：傍晚，公园长椅】  
小亮是我最好的朋友，他决定去山西上大学。夕阳下，我们肩并肩坐着。  
我：“山西的面食可出名了，你这小吃货有福了！”  
小亮笑：“那必须的，说好等我回来，咱俩比赛吃刀削面！”  
我笑着点头，心中却泛起不舍的涟漪。  
【画面渐暗，友情的温暖在空气中弥漫】
回答12：
SMCI可能指的是一家名为Super Micro Computer, Inc.（超微电脑股份有限公司）的公司，它通常被称为Supermicro。该公司成立于1993年，是一家总部位于美国加利福尼亚州圣何塞的计算机硬件和服务器制造商。Supermicro专注于设计、制造和销售高性能服务器技术、存储解决方案和绿色计算系统。它们的产品广泛应用于数据中心、云计算、企业和高性能计算领域，以其高效率和定制化解决方案而知名。


In [41]:
result = memory_service.consolidate_memory()
print(f"consolidate_memory result={result}")

consolidate_memory result=[MEMORY ACTIONS]:
new 用户计划2024年8月9日去北京出差。 (推断时间： 2024年8月9日)
new 用户同学李亚平下个月回上海，用户将与其聚餐。 (推断时间： 2024年9月)


We can see **Memory Consolidation** successfully filtered out fictitious contents, and shows good time sensitivity.

We try more cases to test its resolution of conflicting contents.

In [42]:
response = memory_chat.chat_with_memory(query="今天下午吃什么水果好？")
print("回答13：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="西瓜确实不错，但是我也喜欢吃芒果。我今天想吃芒果。")
print("回答14：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="我最近跳槽去了美团。")
print("回答15：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="我还喜欢吃桃子和苹果。")
print("回答16：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="我不喜欢吃椰子。")
print("回答17：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="我准备下个月去海南冲浪。")
print("回答18：\n" + response.message.content)
response = memory_chat.chat_with_memory(query="明天是我生日。")
print("回答19：\n" + response.message.content)

回答13：
既然您喜欢吃西瓜，今天下午吃西瓜是个不错的选择。
回答14：
那太好了，如果您今天想换换口味，吃芒果也是很好的选择，享受它的香甜吧！
回答15：
恭喜您加入美团！希望您的新工作一切顺利，有新的挑战和机遇。
回答16：
了解了，桃子和苹果都是既营养又美味的选择，多吃水果对身体好，您可以根据季节和个人喜好来挑选。
回答17：
好的，知道您不喜欢椰子，以后在推荐水果时会留意这一点。
回答18：
听起来很刺激！去海南冲浪是个很棒的计划，记得检查当地的天气和海浪情况，做好防晒，祝您玩得开心！
回答19：
生日快乐！提前祝您明天的生日充满快乐和惊喜，希望您有一个难忘的庆祝。


In [ ]:
result = memory_service.consolidate_memory()
print(f"consolidate_memory result={result}")

## Reflection and Re-Consolidation
Now, we have accumulated enough new *observations* in the system, so we can call **Reflection and Re-Consolidation**, let's see what will it get.

In [ ]:
result = memory_service.reflect_and_reconsolidate()
print(f"consolidate_memory result={result}")

## Low response-time (RT) for the user
Here we test the RT of MemoryScope system for the user. Specifically, we test the difference of RT when responding with and without retrieving memory pieces from the system.

In [ ]:
import time

start_time = time.time()
response = memory_chat.chat_with_memory(query="你知道我的乐器爱好是什么吗？")
end_time = time.time()
total_time = end_time - start_time
print("使用记忆检索\n回答20：\n" + response.message.content + f"\n 耗时：{total_time}秒")

start_time = time.time()
response = memory_chat.chat_with_memory(query="你知道我接下去的一个月内有什么计划吗？")
end_time = time.time()
total_time = end_time - start_time
print("使用记忆检索\n回答21：\n" + response.message.content + f"\n 耗时：{total_time}秒")

memory_chat.run_service_operation("delete_all")
start_time = time.time()
response = memory_chat.chat_with_memory(query="你知道我的乐器爱好是什么吗？")
end_time = time.time()
total_time = end_time - start_time
print("使用记忆检索\n回答20：\n" + response.message.content + f"\n 耗时：{total_time}秒")

start_time = time.time()
response = memory_chat.chat_with_memory(query="你知道我接下去的一个月内有什么计划吗？")
end_time = time.time()
total_time = end_time - start_time
print("使用记忆检索\n回答21：\n" + response.message.content + f"\n 耗时：{total_time}秒")